In [31]:
# Load required modules
import csv
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
import dateparser
from dateparser.search import search_dates
import requests
from dateparser import parse
import json
import traceback

from dateparser_data.settings import default_parsers
parsers = [parser for parser in default_parsers if parser != 'relative-time']



In [32]:
#Flashtext algorithm testing

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()

wordsToExclude=[]
stop_words = stopwords.words('english')
numerals=[str(i) for i in range(1,10)]
print(type(numerals[0]))
exclude_words_file=open("exclude-words.txt","r")
exclude_words_lines=exclude_words_file.readlines()

for excludeWord in exclude_words_lines:
    wordsToExclude.append(excludeWord.strip())

wordsToExclude.extend(numerals)
wordsToExclude.extend(stop_words)

setOfWordstoExclude=set(wordsToExclude)
#print(setOfWordstoExclude)
    

file1 = open("extended_placenames.txt","r")

Lines = file1.readlines()
setOfCities=set() 
count = 0
# Strips the newline character
for line in Lines:
    if(line.strip().lower() not in setOfWordstoExclude):
        if(line.strip().lower() not in setOfCities):
            setOfCities.add(line.strip().lower())
            keyword_processor.add_keyword(line.strip().lower())
    if(line.strip().lower()=="big"):
        print(line)
            
        #cities.append(line.strip().lower())
cities=list(setOfCities)
print(len(cities))


<class 'str'>
BIG

1048975


In [33]:
#prepare a keyword list of spider names through the sheet propvided
from flashtext import KeywordProcessor
keyword_processor2 = KeywordProcessor()
file2 = open("spidernames.txt","r")
Lines2=file2.readlines()

for line in Lines2:
    keyword_processor2.add_keyword(line.strip())

In [34]:
def extractDates(text):
    finalDates=[]
    results=search_dates(text,settings={'PARSERS': parsers})
    if(results):
        for result in results:
            dateFields={}
            if(result):
                verbatimDate=result[0]
                if(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})):
                    #print("1")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                    dateFields["day"]=parseDate.day
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})):
                    #print("2")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})):
                    #print("3")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})
                    dateFields["day"]=parseDate.day
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})):
                    #print("4")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})
                    dateFields["year"]=parseDate.year
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})):
                    #print("5")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})
                    dateFields["month"]=parseDate.month
                else:
                    dateFields={}

            if(dateFields):
                finalDates.append(dateFields)                               
        return finalDates

In [35]:
def extract_names(namesArray):
    if(not namesArray):
        return []
        
    names=[]
    for nameObject in namesArray:
        names.append(nameObject['name'])
    return names


In [36]:
def ShortenPermalink(permalink):
   return permalink[0:permalink.find('?')]
    

In [37]:
def ShortenFullLink(fullLink):
    prefix="https://www.facebook.com"
    return (prefix+fullLink[0:fullLink.find('&')])

In [38]:
def GetUniqueId(s):
    numbers = []
    for word in s.split("/"):
       if word.isdigit():
          numbers.append(int(word))
    return numbers[0]


In [39]:
#keywords_found = keyword_processor.extract_keywords("ID please. Nov'2021 Kohima, India.")
#print(keywords_found)

start = time.time()

fields=["Unique id","Author","Desc","Comments Text","Input Text","Locations","Scientific Names(GNRD)","Scientific Names(Flashtext)","Day","Month","Year","Permalink","Thumb","fullLink"]
filename = "Second-cut-curation-sheet-1.csv"

faultyPermalinksFilename="Faulty_Permalinks.csv"
faultyRow=["Author","Desc","Permalink"]

with open(filename, 'a') as csvfile: 
    csvwriter = csv.writer(csvfile)         
    csvwriter.writerow(fields)
    
with open(faultyPermalinksFilename,'a') as faultyCsvFile:
    fcsvwriter=csv.writer(faultyCsvFile)
    fcsvwriter.writerow(faultyRow)

df=pd.read_csv("curation_sample_5.1_1to100.csv")

for i in range(len(df)):
    try:
        author=df.loc[i,"Author"]
        desc=df.loc[i,"Desc"]
        commentsText=df.loc[i,"Comments Text"]
        inputText=df.loc[i,"Input Text"]
        permalink=ShortenPermalink(df.loc[i,"Permalink"])
        uniqueId=GetUniqueId(permalink)
        thumb=df.loc[i,"Thumb"]
        fullLink=df.loc[i,"fullLink"]
        if(type(fullLink)==str):
            fullLink=ShortenFullLink(df.loc[i,"fullLink"])
            
        locations=[]
        names=[]
        namesFromFlashtext=[]
        dates2=[]
        day=""
        month=""
        year=""
        sep=", "
        if(type(inputText)==str):
            locations=keyword_processor.extract_keywords(inputText)
            namesFromFlashtext=keyword_processor2.extract_keywords(inputText)
            dates2=extractDates(inputText)
            days=[]
            months=[]
            years=[]
            
            if(dates2):
                for dateObject in dates2:
                    keys=dateObject.keys()
                    if("year" in keys):
                        years.append(str(dateObject["year"]))
                    if("month" in keys):
                        months.append(str(dateObject["month"]))
                    if("day" in keys):
                        days.append(str(dateObject["day"]))
                
                year=sep.join(years)
                month=sep.join(months)
                day=sep.join(days)
            
            query = {'text':inputText}
            response = requests.get('http://localhost:3006/parse',params=query)
            if(response.json()['names']):
                names=extract_names(response.json()['names'])
            
        row=[uniqueId,author,desc,commentsText,inputText,sep.join(locations),sep.join(names),sep.join(namesFromFlashtext),day,month,year,permalink,thumb,fullLink]
        with open(filename,'a') as csvfile1:
            csvwriter1=csv.writer(csvfile1)
            csvwriter1.writerow(row)
    
    except Exception:
        traceback.print_exc()
        print(f"value of i = {i}")
        print(f"author = {author}")
        print(f"text={desc}")
        print(f"permalink = {permalink}")
        
        errorRow=[author,desc,permalink]
        
        with open(faultyPermalinksFilename,'a') as faultyCsvFile:
            fcsvwriter=csv.writer(faultyCsvFile)
            fcsvwriter.writerow(errorRow)
        
end = time.time()
print(f"Runtime of the program is {end - start}")




Runtime of the program is 96.69486689567566


In [16]:
sampleDates=[{'year': 2032,  'day': 10},{'year': 2021, 'month': 12}]
days=[]
months=[]
years=[]
for dateObject in sampleDates:
    keys=dateObject.keys()
    if("year" in keys):
        years.append(str(dateObject["year"]))
    if("month" in keys):
        months.append(str(dateObject["month"]))
    if("day" in keys):
        days.append(str(dateObject["day"]))
sep=","
print("years=",sep.join(years))
print("momths=",sep.join(months))
print("days=",sep.join(days))
        

years= 2032,2021
momths= 12
days= 10


In [4]:
from dateparser_data.settings import default_parsers
parsers = [parser for parser in default_parsers if parser != 'relative-time']



def extractDates2(text):
    finalDates=[]
    results=search_dates(text,settings={'PARSERS': parsers})
    if(results):
        for result in results:
            dateFields={}
            if(result):
                verbatimDate=result[0]
                if(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})):
                    print("1")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day', 'month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                    dateFields["day"]=parseDate.day
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})):
                    print("2")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month', 'year']})
                    dateFields["year"]=parseDate.year
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})):
                    print("3")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['day','month']})
                    dateFields["day"]=parseDate.day
                    dateFields["month"]=parseDate.month
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})):
                    print("4")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['year']})
                    dateFields["year"]=parseDate.year
                elif(parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})):
                    print("5")
                    parseDate=parse(verbatimDate,settings={'REQUIRE_PARTS': ['month']})
                    dateFields["month"]=parseDate.month
                else:
                    dateFields={}

            if(dateFields):
                finalDates.append(dateFields)                               
        return finalDates

In [33]:
text="nan|ID Pls!!!!!!|Thanks Javed Ahmed for such nice info.!!!!!"
print(extractDates(text))

None


In [61]:
for i in range(1,1000):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
s="https://m.facebook.com/groups/SpiderIndia/permalink/187363884672904/?m_entstream_source=group&refid=18&_ft_=qid.-7358238024638872856%3Amf_story_key.187363884672904%3Atop_level_post_id.187363884672904%3Atl_objid.187363884672904%3Acontent_owner_id_new.100000483133592%3Apage_id.175082879234338%3Asrc.22%3Astory_location.6%3Aattached_story_attachment_style.photo%3Afilter.GroupStoriesByActivityEntQuery%3Aott.AX-oRweiMS84tbhk%3Atds_flgs.3%3Apage_insights.%7B%22175082879234338%22%3A%7B%22page_id%22%3A175082879234338%2C%22page_id_type%22%3A%22group%22%2C%22actor_id%22%3A100000483133592%2C%22dm%22%3A%7B%22isShare%22%3A1%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntGroupMallPostCreationStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A657%2C%22publish_time%22%3A1318526885%2C%22story_name%22%3A%22EntGroupMallPostCreationStory%22%2C%22story_fbid%22%3A%5B187363884672904%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A6%7D%7D&__tn__=-R"
#print(s[0:s.find('?')])
#print(s.split('/'))

numbers = []
for word in s.split("/"):
   if word.isdigit():
      numbers.append(int(word))

print(numbers)

[187363884672904]


In [32]:
s=["aaa","bbb","ccc"]

sep=" | "

print(sep.join(s))

aaa | bbb | ccc


In [46]:
#prepare a keyword list of spider names through the sheet propvided
from flashtext import KeywordProcessor
keyword_processor3 = KeywordProcessor()
keyword_processor3.add_keyword("Prakhar")

s="hello i am Pr aKhar"

ans=keyword_processor3.extract_keywords(s)
print(ans)


[]


In [4]:
s="/photo.php?fbid=289722221053880&id=100000483133592&set=a.289227414436694&source=48&refid=18&_ft_=qid.-7358238024638872856%3Amf_story_key.187363884672904%3Atop_level_post_id.187363884672904%3Atl_objid.187363884672904%3Acontent_owner_id_new.100000483133592%3Apage_id.175082879234338%3Asrc.22%3Astory_location.6%3Aattached_story_attachment_style.photo%3Afilter.GroupStoriesByActivityEntQuery%3Aott.AX-oRweiMS84tbhk%3Atds_flgs.3%3Apage_insights.%7B%22175082879234338%22%3A%7B%22page_id%22%3A175082879234338%2C%22page_id_type%22%3A%22group%22%2C%22actor_id%22%3A100000483133592%2C%22dm%22%3A%7B%22isShare%22%3A1%2C%22originalPostOwnerID%22%3A0%7D%2C%22psn%22%3A%22EntGroupMallPostCreationStory%22%2C%22post_context%22%3A%7B%22object_fbtype%22%3A657%2C%22publish_time%22%3A1318526885%2C%22story_name%22%3A%22EntGroupMallPostCreationStory%22%2C%22story_fbid%22%3A%5B187363884672904%5D%7D%2C%22role%22%3A1%2C%22sl%22%3A6%7D%7D&__tn__=EHH-R"
ans=ShortenFullLink(s)
print(ans)

https://www.facebook.com//photo.php?fbid=289722221053880
